# Porównanie wszystkich modeli: Graph Transformers vs Baselines vs Hybrid

Ten notebook porównuje wszystkie zaimplementowane modele:

## Graph Transformers
1. **GOAT** - Global attention z virtual nodes (O(N))
2. **Exphormer** - Sparse attention z expander graphs (O(Nd))

## Baseline GNNs
3. **GCN** - Graph Convolutional Network (O(E))
4. **GAT** - Graph Attention Network (O(E))
5. **GIN** - Graph Isomorphism Network (O(E)) - **NOWY**
6. **GraphMLP** - MLP bez struktury grafu

## Modele Hybrydowe
7. **GCNVirtualNode** - GCN z Virtual Node (O(E+N)) - **NOWY**
8. **GINVirtualNode** - GIN z Virtual Node (O(E+N)) - **NOWY**

---

## Datasety
- **ZINC** (regresja, MAE)
- **ogbg-molhiv** (klasyfikacja binarna, ROC-AUC)
- **ogbg-molpcba** (multi-label, AP) - **NOWY**

In [5]:
# Setup - dodaj ścieżkę do projektu
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

from torch_geometric.loader import DataLoader
from tqdm.auto import tqdm

# Importuj wszystkie modele
from models import (
    GOAT, Exphormer,           # Graph Transformers
    GCN, GAT, GIN, GraphMLP,   # Baselines
    GCNVirtualNode, GINVirtualNode  # Hybrid
)

# Importuj utilities
from src.utils.data import (
    load_zinc_dataset, 
    load_molhiv_dataset, 
    load_molpcba_dataset
)
from src.utils.positional_encodings import precompute_positional_encodings
from src.utils.complexity import ComplexityTracker, count_parameters
from src.utils.metrics import compute_metrics

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Device: cpu
PyTorch version: 2.8.0


## Konfiguracja eksperymentu

**Wybierz tryb:**
- `cpu` - szybki test (mały dataset, mało epok) - ~10-15 minut
- `gpu` - pełny eksperyment (cały dataset, więcej epok) - ~1-2 godziny

In [6]:
# ========================================================
# KONFIGURACJA - ZMIEŃ TUTAJ
# ========================================================

EXPERIMENT_MODE = "cpu"  # "cpu" lub "gpu"

# ========================================================

if EXPERIMENT_MODE == "cpu":
    # Tryb CPU - szybki test
    CONFIG = {
        'dataset': 'zinc',
        'use_subset': True,
        'subset_size': 500,
        'batch_size': 32,
        'num_epochs': 10,
        'hidden_dim': 64,
        'num_layers': 3,
        'num_heads': 4,
        'lr': 1e-3,
        'dropout': 0.1,
        'pe_dim': 8,
        'device': 'cpu',
    }
    print("🖥️  TRYB CPU - Szybki test")
else:
    # Tryb GPU - pełny eksperyment
    CONFIG = {
        'dataset': 'zinc',
        'use_subset': False,
        'subset_size': None,
        'batch_size': 64,
        'num_epochs': 100,
        'hidden_dim': 256,
        'num_layers': 5,
        'num_heads': 8,
        'lr': 1e-4,
        'dropout': 0.1,
        'pe_dim': 16,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    }
    print("🚀 TRYB GPU - Pełny eksperyment")

print(f"\nKonfiguracja:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

🖥️  TRYB CPU - Szybki test

Konfiguracja:
  dataset: zinc
  use_subset: True
  subset_size: 500
  batch_size: 32
  num_epochs: 10
  hidden_dim: 64
  num_layers: 3
  num_heads: 4
  lr: 0.001
  dropout: 0.1
  pe_dim: 8
  device: cpu


## Ładowanie datasetu

In [7]:
def load_dataset_by_name(name, use_subset=False, subset_size=500, pe_dim=8):
    """Załaduj dataset po nazwie."""
    print(f"Ładowanie datasetu: {name}")
    
    if name == 'zinc':
        dataset, split_idx = load_zinc_dataset()
        task_type = 'regression'
        metric = 'mae'
        out_channels = 1
        in_channels = dataset[0].x.shape[1] if dataset[0].x.dim() > 1 else 1
    elif name == 'molhiv':
        dataset, split_idx = load_molhiv_dataset()
        task_type = 'binary_classification'
        metric = 'rocauc'
        out_channels = 1
        in_channels = dataset[0].x.shape[1]
    elif name == 'molpcba':
        dataset, split_idx = load_molpcba_dataset()
        task_type = 'multi_label'
        metric = 'ap'
        out_channels = 128  # 128 tasks
        in_channels = dataset[0].x.shape[1]
    else:
        raise ValueError(f"Nieznany dataset: {name}")
    
    # Użyj podzbioru jeśli potrzeba
    if use_subset:
        print(f"Używam podzbioru: {subset_size} grafów")
        indices = torch.randperm(len(dataset))[:subset_size]
        # Stwórz nowe splity
        train_size = int(0.8 * subset_size)
        val_size = int(0.1 * subset_size)
        split_idx = {
            'train': indices[:train_size],
            'valid': indices[train_size:train_size+val_size],
            'test': indices[train_size+val_size:],
        }
    
    # Precompute positional encodings
    print("Obliczam positional encodings...")
    dataset = precompute_positional_encodings(dataset, pe_type='laplacian', pe_dim=pe_dim)
    
    print(f"Dataset załadowany:")
    print(f"  Total: {len(dataset)} grafów")
    print(f"  Train: {len(split_idx['train'])}, Val: {len(split_idx['valid'])}, Test: {len(split_idx['test'])}")
    print(f"  In channels: {in_channels}, Out channels: {out_channels}")
    print(f"  Task type: {task_type}, Metric: {metric}")
    
    return dataset, split_idx, {
        'task_type': task_type,
        'metric': metric,
        'in_channels': in_channels,
        'out_channels': out_channels,
    }

# Załaduj dataset
dataset, split_idx, dataset_info = load_dataset_by_name(
    CONFIG['dataset'],
    use_subset=CONFIG['use_subset'],
    subset_size=CONFIG['subset_size'],
    pe_dim=CONFIG['pe_dim'],
)

Ładowanie datasetu: zinc


Extracting data/molecules.zip
Processing...
Processing test dataset: 100%|██████████| 1000/1000 [00:00<00:00, 27112.68it/s]
Done!


Używam podzbioru: 500 grafów
Obliczam positional encodings...
Setting up laplacian positional encodings (dim=8)...
Precomputing encodings...
  Processed 100/10000 graphs
  Processed 200/10000 graphs
  Processed 300/10000 graphs
  Processed 400/10000 graphs
  Processed 500/10000 graphs
  Processed 600/10000 graphs
  Processed 700/10000 graphs
  Processed 800/10000 graphs
  Processed 900/10000 graphs
  Processed 1000/10000 graphs
  Processed 1100/10000 graphs
  Processed 1200/10000 graphs
  Processed 1300/10000 graphs
  Processed 1400/10000 graphs
  Processed 1500/10000 graphs
  Processed 1600/10000 graphs
  Processed 1700/10000 graphs
  Processed 1800/10000 graphs
  Processed 1900/10000 graphs
  Processed 2000/10000 graphs
  Processed 2100/10000 graphs
  Processed 2200/10000 graphs
  Processed 2300/10000 graphs
  Processed 2400/10000 graphs
  Processed 2500/10000 graphs
  Processed 2600/10000 graphs
  Processed 2700/10000 graphs
  Processed 2800/10000 graphs
  Processed 2900/10000 graph

In [8]:
# Stwórz DataLoadery
train_loader = DataLoader(
    dataset[split_idx['train']], 
    batch_size=CONFIG['batch_size'], 
    shuffle=True
)
val_loader = DataLoader(
    dataset[split_idx['valid']], 
    batch_size=CONFIG['batch_size'], 
    shuffle=False
)
test_loader = DataLoader(
    dataset[split_idx['test']], 
    batch_size=CONFIG['batch_size'], 
    shuffle=False
)

print(f"Batchy: Train={len(train_loader)}, Val={len(val_loader)}, Test={len(test_loader)}")

Batchy: Train=13, Val=2, Test=2


## Definicja modeli

In [10]:
def create_models(config, dataset_info):
    """Stwórz wszystkie modele do porównania."""
    in_ch = dataset_info['in_channels']
    out_ch = dataset_info['out_channels']
    hidden = config['hidden_dim']
    layers = config['num_layers']
    heads = config['num_heads']
    dropout = config['dropout']
    pe_dim = config['pe_dim']
    
    models = {
        # ===== Baselines =====
        'GCN': GCN(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            dropout=dropout,
        ),
        'GAT': GAT(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            num_heads=heads,
            dropout=dropout,
        ),
        'GIN': GIN(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            dropout=dropout,
        ),
        'GraphMLP': GraphMLP(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            dropout=dropout,
        ),
        
        # ===== Hybrid Models =====
        'GCN+VN': GCNVirtualNode(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            dropout=dropout,
        ),
        'GIN+VN': GINVirtualNode(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            dropout=dropout,
        ),
        
        # ===== Graph Transformers =====
        'GOAT': GOAT(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            num_heads=heads,
            pe_dim=pe_dim,
            dropout=dropout,
        ),
        'Exphormer': Exphormer(
            in_channels=in_ch,
            hidden_channels=hidden,
            out_channels=out_ch,
            num_layers=layers,
            num_heads=heads,
            pe_dim=pe_dim,
            dropout=dropout,
        ),
    }
    
    return models

# Stwórz modele
models = create_models(CONFIG, dataset_info)

# Pokaż liczbę parametrów
print("\n" + "="*60)
print("MODELE I LICZBA PARAMETRÓW")
print("="*60)
for name, model in models.items():
    params = count_parameters(model)
    # count_parameters zwraca dict z kluczami: 'total', 'trainable', etc.
    print(f"{name:15s}: {params['total']:>10,} parametrów")
print("="*60)


MODELE I LICZBA PARAMETRÓW


TypeError: unsupported format string passed to dict.__format__

## Funkcje treningowe

In [ ]:
def train_epoch(model, loader, optimizer, device, task_type='regression'):
    """Trenuj przez jedną epokę."""
    model.train()
    total_loss = 0
    
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        
        out = model(batch)
        
        # Oblicz loss w zależności od typu zadania
        if task_type == 'regression':
            y = batch.y.float().view(-1, 1)
            loss = F.mse_loss(out, y)
        elif task_type == 'binary_classification':
            y = batch.y.float().view(-1, 1)
            loss = F.binary_cross_entropy_with_logits(out, y)
        elif task_type == 'multi_label':
            y = batch.y.float()
            # Ignoruj NaN labels
            mask = ~torch.isnan(y)
            loss = F.binary_cross_entropy_with_logits(out[mask], y[mask])
        else:
            raise ValueError(f"Unknown task type: {task_type}")
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        total_loss += loss.item() * batch.num_graphs
    
    return total_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(model, loader, device, task_type='regression', metric='mae'):
    """Ewaluuj model."""
    model.eval()
    
    all_preds = []
    all_labels = []
    total_loss = 0
    
    for batch in loader:
        batch = batch.to(device)
        out = model(batch)
        
        if task_type == 'regression':
            y = batch.y.float().view(-1, 1)
            loss = F.mse_loss(out, y)
            all_preds.append(out.cpu())
            all_labels.append(y.cpu())
        elif task_type == 'binary_classification':
            y = batch.y.float().view(-1, 1)
            loss = F.binary_cross_entropy_with_logits(out, y)
            all_preds.append(torch.sigmoid(out).cpu())
            all_labels.append(y.cpu())
        elif task_type == 'multi_label':
            y = batch.y.float()
            mask = ~torch.isnan(y)
            loss = F.binary_cross_entropy_with_logits(out[mask], y[mask])
            all_preds.append(torch.sigmoid(out).cpu())
            all_labels.append(y.cpu())
        
        total_loss += loss.item() * batch.num_graphs
    
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)
    
    # Oblicz metrykę
    if metric == 'mae':
        score = F.l1_loss(all_preds, all_labels).item()
    elif metric == 'rocauc':
        from sklearn.metrics import roc_auc_score
        try:
            score = roc_auc_score(all_labels.numpy(), all_preds.numpy())
        except:
            score = 0.5
    elif metric == 'ap':
        from sklearn.metrics import average_precision_score
        mask = ~torch.isnan(all_labels)
        try:
            score = average_precision_score(all_labels[mask].numpy(), all_preds[mask].numpy())
        except:
            score = 0.0
    else:
        score = total_loss / len(loader.dataset)
    
    return score, total_loss / len(loader.dataset)


def train_model(model, train_loader, val_loader, config, dataset_info, verbose=True):
    """Pełny trening modelu."""
    device = torch.device(config['device'])
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config['num_epochs']
    )
    
    task_type = dataset_info['task_type']
    metric = dataset_info['metric']
    
    # Czy wyższy wynik jest lepszy?
    higher_is_better = metric in ['rocauc', 'ap', 'accuracy']
    
    best_val_score = float('-inf') if higher_is_better else float('inf')
    history = {'train_loss': [], 'val_score': []}
    
    # Tracking czasu
    start_time = time.time()
    
    pbar = tqdm(range(config['num_epochs']), disable=not verbose)
    for epoch in pbar:
        train_loss = train_epoch(model, train_loader, optimizer, device, task_type)
        val_score, val_loss = evaluate(model, val_loader, device, task_type, metric)
        
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['val_score'].append(val_score)
        
        # Update best
        if higher_is_better:
            if val_score > best_val_score:
                best_val_score = val_score
        else:
            if val_score < best_val_score:
                best_val_score = val_score
        
        pbar.set_postfix({
            'loss': f'{train_loss:.4f}',
            f'val_{metric}': f'{val_score:.4f}',
            f'best': f'{best_val_score:.4f}'
        })
    
    train_time = time.time() - start_time
    
    return {
        'best_val_score': best_val_score,
        'history': history,
        'train_time': train_time,
    }

## Trening wszystkich modeli

In [ ]:
# Trenuj wszystkie modele
results = {}

print("="*70)
print("TRENING WSZYSTKICH MODELI")
print("="*70)

for name, model in models.items():
    print(f"\n{'='*70}")
    print(f"Trenuję: {name}")
    print(f"{'='*70}")
    
    # Reset modelu (świeże wagi)
    for layer in model.modules():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()
    
    # Mierz pamięć
    if torch.cuda.is_available() and CONFIG['device'] == 'cuda':
        torch.cuda.reset_peak_memory_stats()
    
    # Trenuj
    result = train_model(
        model, 
        train_loader, 
        val_loader, 
        CONFIG, 
        dataset_info,
        verbose=True
    )
    
    # Zapisz wyniki
    result['params'] = count_parameters(model)['total']
    if torch.cuda.is_available() and CONFIG['device'] == 'cuda':
        result['peak_memory_mb'] = torch.cuda.max_memory_allocated() / 1e6
    else:
        result['peak_memory_mb'] = 0
    
    results[name] = result
    
    print(f"\n✓ {name}: Best val {dataset_info['metric']} = {result['best_val_score']:.4f}")
    print(f"  Czas: {result['train_time']:.1f}s, Parametry: {result['params']:,}")

print("\n" + "="*70)
print("TRENING ZAKOŃCZONY")
print("="*70)

## Podsumowanie wyników

In [ ]:
# Tabela wyników
import pandas as pd

metric = dataset_info['metric']
higher_is_better = metric in ['rocauc', 'ap', 'accuracy']

summary = []
for name, res in results.items():
    summary.append({
        'Model': name,
        f'Val {metric.upper()}': res['best_val_score'],
        'Parametry': res['params'],
        'Czas (s)': res['train_time'],
        'Pamięć (MB)': res['peak_memory_mb'],
    })

df = pd.DataFrame(summary)

# Sortuj według metryki
df = df.sort_values(f'Val {metric.upper()}', ascending=not higher_is_better)

print("\n" + "="*80)
print(f"PODSUMOWANIE WYNIKÓW - Dataset: {CONFIG['dataset'].upper()}")
print("="*80)
print(df.to_string(index=False))
print("="*80)

In [ ]:
# Wykresy
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Porównanie metryk
ax = axes[0]
names = list(results.keys())
scores = [results[n]['best_val_score'] for n in names]
colors = ['#2ecc71' if 'VN' in n else '#3498db' if n in ['GOAT', 'Exphormer'] else '#95a5a6' for n in names]
bars = ax.barh(names, scores, color=colors)
ax.set_xlabel(f'Val {metric.upper()}')
ax.set_title(f'Porównanie modeli ({CONFIG["dataset"]})')
for bar, score in zip(bars, scores):
    ax.text(bar.get_width(), bar.get_y() + bar.get_height()/2, 
            f' {score:.4f}', va='center', fontsize=9)

# 2. Czas treningu
ax = axes[1]
times = [results[n]['train_time'] for n in names]
ax.barh(names, times, color=colors)
ax.set_xlabel('Czas treningu (s)')
ax.set_title('Czas treningu')

# 3. Parametry vs Wynik
ax = axes[2]
params = [results[n]['params'] for n in names]
for i, name in enumerate(names):
    ax.scatter(params[i], scores[i], s=100, c=colors[i], label=name)
ax.set_xlabel('Liczba parametrów')
ax.set_ylabel(f'Val {metric.upper()}')
ax.set_title('Parametry vs Wynik')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nWykres zapisany: model_comparison.png")

In [ ]:
# Krzywe uczenia
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, (name, res) in enumerate(results.items()):
    if idx >= len(axes):
        break
    ax = axes[idx]
    epochs = range(1, len(res['history']['train_loss']) + 1)
    
    ax.plot(epochs, res['history']['train_loss'], label='Train Loss', alpha=0.7)
    ax2 = ax.twinx()
    ax2.plot(epochs, res['history']['val_score'], 'r-', label=f'Val {metric}', alpha=0.7)
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss', color='b')
    ax2.set_ylabel(f'{metric.upper()}', color='r')
    ax.set_title(f'{name}\nBest: {res["best_val_score"]:.4f}')
    ax.grid(True, alpha=0.3)

# Ukryj puste subploty
for idx in range(len(results), len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.savefig('learning_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nWykres zapisany: learning_curves.png")

## Wnioski

Po uruchomieniu eksperymentu, wypełnij wnioski:

1. **Najlepszy model:** `[TUTAJ]`
2. **Czy modele hybrydowe (VN) poprawiają wyniki?** `[TAK/NIE]`
3. **Czy Graph Transformers są lepsze od GNN?** `[TAK/NIE/ZALEŻY]`
4. **Trade-off czas/jakość:** `[OPIS]`

In [ ]:
# Zapisz wyniki do pliku
import json

# Przygotuj wyniki do zapisu
save_results = {}
for name, res in results.items():
    save_results[name] = {
        'best_val_score': float(res['best_val_score']),
        'params': res['params'],
        'train_time': res['train_time'],
        'peak_memory_mb': res['peak_memory_mb'],
    }

output = {
    'config': CONFIG,
    'dataset_info': dataset_info,
    'results': save_results,
}

filename = f"results_{CONFIG['dataset']}_{EXPERIMENT_MODE}.json"
with open(filename, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Wyniki zapisane do: {filename}")